In [1]:
import pandas as pd
import os
import numpy as np
import json

In [2]:
temp=[]
csvs = ['purchase_agree.csv', 'divorce.csv']
#for csv in os.listdir('./csv'):
for csv in csvs:
    temp.append(pd.read_csv('./csv/'+csv))
df = pd.concat(temp, ignore_index=True)
df.shape

(399, 3)

In [3]:
df

,ADR REFERENCE NUMBER,PAGE NO,QC VALUE
0,ADR-2022-100004176625,1,Purchase Agreement
1,ADR-2022-100004179066,1,Purchase Agreement
2,ADR-2022-100004181773,1,Purchase Agreement
3,ADR-2022-200004178677,1,Purchase Agreement
4,ADR-2022-200004179727,1,Purchase Agreement
...,...,...,...
394,ADR-2022-100004188992,"20,21,22",Divorce Decree
395,ADR-2022-100004190135,"1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1...",Divorce Decree
396,ADR-2022-100004189270,"1,2,3",Divorce Decree
397,ADR-2022-200004191074,"1,2,3,4",Divorce Decree


In [4]:
df['QC VALUE'].value_counts()

Purchase Agreement    346
Divorce Decree         52
Flood Certificate       1
Name: QC VALUE, dtype: int64

In [5]:
def get_text(ADR,pageno):
    data = json.load(open('./ocr/'+ADR+'.txt', encoding="utf-8"))
    txt = ''
    annotations=[]
    for page in data['analyzeResult']['readResults']:
        if page['page']==pageno:
            for line in page['lines']:
                txt += line['text']+' '
    return(txt)

In [6]:
category = []
text = []
adr = []
for index,row in df.iterrows():
    ADR = row['ADR REFERENCE NUMBER']
    if ADR+'.txt' in os.listdir('./ocr'):
        #print(ADR)
        for page in row['PAGE NO'].split(','):
            category.append(row['QC VALUE'])
            adr.append(ADR)
            try:
                text.append(get_text(ADR,int(page)))
            except:
                text.append(np.nan)

In [14]:
data = dict()
data['ADR'] = adr
data['text'] = text
data['category'] = category

In [15]:
data = pd.DataFrame(data)
data.head()

,ADR,text,category
0,ADR-2022-100004176625,DocuSign Envelope ID: 4EAE2023-993D-4921-9F40-...,Purchase Agreement
1,ADR-2022-100004179066,dotloop signature verification: dtlp.us/9ZlB-L...,Purchase Agreement
2,ADR-2022-100004181773,PURCHASE AGREEMENT Buyer: Tyrone Jennings Sell...,Purchase Agreement
3,ADR-2022-200004178677,GENERAL RESIDENTIAL SALES CONTRACT Form Approv...,Purchase Agreement
4,ADR-2022-200004179727,DocuSign Envelope ID: E3CAA5F7-7861-4BAB-B895-...,Purchase Agreement


In [16]:
data.isnull().sum()

ADR            0
text        3101
category       0
dtype: int64

In [17]:
data.dropna(inplace=True)
data.shape

(3611, 3)

In [18]:
data['category'].value_counts()

Purchase Agreement    3271
Divorce Decree         340
Name: category, dtype: int64

In [19]:
data.to_csv('./data_demo2.csv', header=True, index=False)

In [22]:
df1 = pd.read_csv('./data_demo.csv')
df1.shape

(20144, 3)

In [23]:
df2 = pd.read_csv('./data_demo2.csv')
df2.shape

(3611, 3)

In [25]:
df = pd.concat([df1,df2])
df.shape

(23755, 3)

In [26]:
df.head()

,ADR,text,category
0,ADR-2022-100004022857,"# Lee, Terrilynn, Marie 000029128 I=0000 29128...",Award Letter(s)
1,ADR-2022-100004025366,BeBenefits A Service of the Department of Vete...,Award Letter(s)
2,ADR-2022-100004026667,Service Verification Letter DEPARTMENT OF VETE...,Award Letter(s)
3,ADR-2022-100004027737,AB8 Benefit Verification Letter DEPARTMENT OF ...,Award Letter(s)
4,ADR-2022-100004029343,The Prudential Insurance Company of America Pr...,Award Letter(s)


In [28]:
df.to_csv('./data.csv', header=True, index=False)